In [ ]:
%pip install bertopic

In [ ]:
%pip install top2vec

  Using cached top2vec-1.0.34-py3-none-any.whl (27 kB)


In [ ]:
%%python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 27.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=0.8)
sns.set_palette('tab10')
%matplotlib inline
import holidays
from wordcloud import WordCloud
import re
from collections import Counter
import spacy
import time

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance

from keras.preprocessing.text import Tokenizer
import torch.nn as nn
from keras.utils import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from top2vec import Top2Vec

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def f(df):

  def set_sentiment(value):
    if value < 3:
        value = "Negative"
    elif value > 3:
        value = "Positive"
    else:
        value = "Neutral"
    return value
  df["Sentiment"] = df["Rating"].apply(lambda x: set_sentiment(x))

  df["Length"] = df["Comment"].str.len()

  # Drop missing values (comments)
  df.dropna(subset='Comment', inplace=True)

  # Number of words column
  df["nb_words"] = df["Comment"].apply(lambda x: len(x.split(" ")))

  df = df[df["Sentiment"]!="Neutral"]
  df.reset_index(inplace=True, drop=True)


  def replace_punctuation(comment):
    if isinstance(comment, str):
      return ''.join([' ' if c in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~' else c for c in comment])
    else:
      return comment

  # Apply the punctuation replacement function to the "Comment" and "Reply" column for both Eni and TotalEnergies

  def preprocess_column(df, column):
      df[column] = df[column].apply(replace_punctuation)
      df[column] = df[column].str.lower()

  preprocess_column(df, 'Comment')

  # Load a pre-trained language model (e.g., French)
  nlp = spacy.load("fr_core_news_sm")

  # Process text
  def tokenize_lemm_func(text):
      # Analyze text with spacy
      doc = nlp(text)

      # Lemmatization and stop words removal
      lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop]

      # Join the tokenized words as a sequence to form back the processed text
      lemmatized_text = ' '.join(lemmatized_tokens)

      return lemmatized_text

  df["Comment_clean"] = df['Comment'].apply(tokenize_lemm_func)

  # Function to remove one-character words
  def remove_one_char_words(text):
    # Split the text into words
    words = text.split()

    # Filter out one-character words
    filtered_words = [word for word in words if len(word) > 1]

    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)

    return filtered_text

  # Apply the function to the 'Text' column
  df['Comment_clean'] = df['Comment_clean'].apply(remove_one_char_words)

  df["Split"] = df['Comment_clean'].apply(lambda x:str(x).split())

  df_pos = df[df["Sentiment"]=="Positive"]
  df_neg = df[df["Sentiment"]=="Negative"]

  return df_pos, df_neg

In [ ]:
df = pd.read_csv("/content/Reviews_totalenergies.csv")

df_pos, df_neg = f(df)

In [ ]:
len(df)

905

### DF_neg Total energy

In [ ]:
umap_args = {'n_neighbors': 10,
             'n_components': 5,
             'metric': 'cosine',
             "random_state": 42}
hdbscan_args = {'min_cluster_size': 10,
                    'min_samples':5,
                    'metric': 'euclidean',
                    'cluster_selection_method': 'eom'}
topic_model = Top2Vec(
        documents= df_neg["Comment_clean"].to_list(),
        speed='deep-learn',
        workers=8,
        min_count = 0,
        embedding_model='distiluse-base-multilingual-cased',
        umap_args = umap_args,
        hdbscan_args = hdbscan_args)

topic_words, word_scores, topic_nums = topic_model.get_topics(10)
for topic in topic_nums:
    topic_model.generate_topic_wordcloud(topic)


In [ ]:
topic_model.hierarchical_topic_reduction(num_topics=3)

[[4, 5, 1], [2, 6, 7, 0], [9, 8, 3]]

In [ ]:
[[5, 6, 0, 7], [4, 1, 2], [9, 8, 3]]

In [ ]:
df_neg["Topic"] = topic_model.doc_top

macro_topic = [[5, 6, 0, 7], [4, 1, 2], [9, 8, 3]]

mapping_dict = {}
for index, lst in enumerate(macro_topic):
    for digit in lst:
        mapping_dict[digit] = index

# Map the values in the "Topic" column
df_neg['Topic'] = df_neg['Topic'].map(mapping_dict)

df2 = pd.DataFrame({'Comment': df_neg['Comment_clean'],  'Topic': df_neg['Topic']})

map_top = {0: "Telephone/Raport au Client/Mail&Spam/Chatbot/Inexplicable/Injoignable", 1: "Frauduleux/Indeminisation/plainte", 2:"Energie/Gaz/Facture/Prix/Problème de Débit..." }

df2['Topic'] = df2['Topic'].map(map_top)

df2.groupby('Topic').size().reset_index(name='Count')

,Topic,Count
0,Energie/Gaz/Facture/Prix/Problème de Débit...,82
1,Frauduleux/Indeminisation/plainte,134
2,Telephone/Raport au Client/Mail&Spam/Chatbot/I...,138


# DF_pos Total Energy

In [ ]:
umap_args = {'n_neighbors': 10,
             'n_components': 5,
             'metric': 'cosine',
             "random_state": 42}
hdbscan_args = {'min_cluster_size': 10,
                    'min_samples':5,
                    'metric': 'euclidean',
                    'cluster_selection_method': 'eom'}
topic_model = Top2Vec(
        documents= df_pos["Comment_clean"].to_list(),
        speed='deep-learn',
        workers=8,
        min_count = 0,
        embedding_model='distiluse-base-multilingual-cased',
        umap_args = umap_args,
        hdbscan_args = hdbscan_args)

topic_words, word_scores, topic_nums = topic_model.get_topics(12)
for topic in topic_nums:
    topic_model.generate_topic_wordcloud(topic)

In [ ]:
df_pos["Topic"] = topic_model.doc_top

macro_topic = topic_model.hierarchical_topic_reduction(num_topics=3)

mapping_dict = {}
for index, lst in enumerate(macro_topic):
    for digit in lst:
        mapping_dict[digit] = index

# Map the values in the "Topic" column
df_pos['Topic'] = df_pos['Topic'].map(mapping_dict)

df3 = pd.DataFrame({'Comment': df_pos['Comment'],  'Topic': df_pos['Topic']})

map_top = {0: "Energie/mensuel/Le service en lui même", 1: "Facile/Rapide/Accessible/Efficace/Competitif/Avantageux", 2: "Gentil/Bienveillant/Sympa/Agreable"}

df3['Topic'] = df3['Topic'].map(map_top)

df3.groupby('Topic').size().reset_index(name='Count')


,Topic,Count
0,Energie/mensuel/Le service en lui même,202
1,Facile/Rapide/Accessible/Efficace/Competitif/A...,205
2,Gentil/Bienveillant/Sympa/Agreable,130


# DF concurent

In [ ]:
df = pd.read_csv("/content/fournisseurs_energie_top5_forums_clean.csv")

In [ ]:
df.rename(columns={'note': 'Rating', "text": "Comment"}, inplace=True)

In [ ]:
df_engie = df[(df["fournisseur"].str[-8:-3]=="engie")|(df["fournisseur"].str[-21:-16]=="engie")]
df_ohm_energie = df[df["fournisseur"].str[-15:-4]=="ohm-energie"]
df_edf = df[(df["fournisseur"].str[-6:-3]=="edf") | (df["fournisseur"].str[-10:-4]=="edfenr")]
df_eni = df[df["fournisseur"].str[-6:-3]=="eni"]
df_ekwateur = df[df["fournisseur"].str[-11:-3]=="ekwateur"]
df_sowee = df[df["fournisseur"].str[-8:-3]=="sowee"]
df_happe = df[df["fournisseur"].str[-9:-3]=="happ-e"]